In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop,Adam
from keras import callbacks
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from keras.models import model_from_json

In [ ]:
print("resize images in the train folder")
folder ="train/clean/"
#copy_folder="images_r/train/nonwaste/"
w = int(224)
h = int(224)
for i in os.listdir(folder):
    file = f"{folder}/{i}"
    im = Image.open(file)
    im = im.resize((w, h), Image.ANTIALIAS)
    im.save(file) 
   
print("resize images in the train folder")
folder ="train/waste/"
#copy_folder="images_r/train/nonwaste/"
w = int(224)
h = int(224)
for i in os.listdir(folder):
    file = f"{folder}/{i}"
    im = Image.open(file)
    im = im.resize((w, h), Image.ANTIALIAS)
    im.save(file)
    
print("resize images in the val folder")
folder ="validation/waste/"
#copy_folder="images_r/train/nonwaste/"
w = int(224)
h = int(224)
for i in os.listdir(folder):
    file = f"{folder}/{i}"
    im = Image.open(file)
    im = im.resize((w, h), Image.ANTIALIAS)
    im.save(file)     

print("resize images in the val folder")
folder ="validation/clean/"
#copy_folder="images_r/train/nonwaste/"
w = int(224)
h = int(224)
for i in os.listdir(folder):
    file = f"{folder}/{i}"
    im = Image.open(file)
    im = im.resize((w, h), Image.ANTIALIAS)
    im.save(file)     

In [ ]:

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [ ]:
validation = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

In [ ]:
train_ds = train_datagen.flow_from_directory('train/',target_size=(224,224),batch_size=3,class_mode='binary')


In [ ]:
validation_ds = validation.flow_from_directory('validation/',target_size=(224,224),batch_size=3,class_mode='binary')
#test_ds = validation.flow_from_directory('validation/',target_size=(224,224),batch_size=3,class_mode='binary')

In [ ]:
train_ds.class_indices

In [ ]:
train_ds.classes

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(224,224,3)),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation='relu'),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy']
             )

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                    mode ="min", patience = 5, 
                                       restore_best_weights = True)

In [ ]:
model_fit = model.fit(train_ds,
                   epochs=30,batch_size=15,
                     validation_data=validation_ds, callbacks =[earlystopping])

In [ ]:
print(model_fit.history.keys())

In [ ]:

plt.plot(model_fit.history['accuracy'])
plt.plot(model_fit.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(model_fit.history['loss'])
plt.plot(model_fit.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


In [ ]:
test_generator = ImageDataGenerator()

test_ds = test_generator.flow_from_directory(directory=r"./test/",
                                                    target_size=(224, 224),
                                                    batch_size=1,
                                                    class_mode='binary', shuffle=False)

In [ ]:
filenames = test_ds.filenames
nb_samples = len(filenames)

Y_pred = model.predict_generator(test_ds,steps=np.ceil(test_ds.samples/test_ds.batch_size),verbose=1,workers=0)
#y_pred = np.argmax(Y_pred, axis=1)
y_pred = np.where(Y_pred>0.5,1,0)
#print(y_pred)
print('Confusion Matrix')
print(confusion_matrix(test_ds.classes, y_pred))
print('Classification Report')
target_names = test_ds.classes
class_labels = list(test_ds.class_indices.keys()) 

report = classification_report(target_names, y_pred, target_names=class_labels)
print(report)

In [ ]:
dir_path = 'test/waste'
for i in os.listdir(dir_path):
  img = image.load_img(dir_path+'//' + i,target_size=(224,224))
  plt.imshow(img)
  plt.show()

  X=image.img_to_array(img)
  X=np.expand_dims(X,axis=0)
  images = np.vstack([X])
  val = model.predict(images)
  if val == 0:
    print("The given image is clean")
  else:
    print("The given image is waste")  

In [ ]:
model_json_50_84 = model.to_json()
with open("model50_84.json", "w") as json_file:
    json_file.write(model_json_50_84)
# serialize weights to HDF5
model.save_weights("cnn50_84.h5")
print("Saved model.")

In [ ]:
'''
json_file = open('model32_98.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("cnn32_87.h5")
print("Loaded model")
'''

In [ ]:
#loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])